In [1]:
import json
import re
from string import punctuation
import glob
import pandas as pd
import os

In [76]:
def is_sentence(text: str):
    sentence_pattern = r'.*[.!?]$'
    return bool(re.search(sentence_pattern, text.strip()))

def clean(text: str):
    punctuation_pattern = re.compile(f"[{punctuation}]")
    spaces_pattern = re.compile(r"\s+")
    non_ascii_pattern = re.compile(r"[^\x00-\x7F\u0E00-\u0E7F]")
    text = (
        text.replace("'t", " not ")
        .replace("’t", " not ")
        .replace("'s", " is ")
        .replace("’s", " is ")
        .replace("'re", " are ")
        .replace("’re", " are ")
        .replace("'m", " am ")
        .replace("’m", " am ")
        .replace("'ll", " will ")
        .replace("’ll", " will ")
        .replace("\n", " ")
    )
    text = re.sub(punctuation_pattern, " ", text)
    text = re.sub(non_ascii_pattern, " ", text)
    return re.sub(spaces_pattern, " ", text).strip()

def format(data):
    thai_sentences = list()
    eng_sentences = list()
    thai_sentence = ''
    eng_sentence = ''
    j = 0
    if ('th' in data['transcript'] and 'en' in data['transcript']):
        for i, th in enumerate(data['transcript']['th']):
            if (j >= len(data['transcript']['en'])): break
            en = data['transcript']['en'][j]
            en_text = en['text']
            en_start = en['start']  
            en_duration = en['duration']
            th_text = th['text']
            th_start = th['start']
            th_duration = th['duration']
            if (abs(en_start - th_start) < 0.5):
                thai_sentence += ' ' + th_text
                eng_sentence += ' ' + en_text
                if (is_sentence(eng_sentence)):
                    eng_sentences.append(clean(eng_sentence))
                    thai_sentences.append(clean(thai_sentence))
                    thai_sentence = ''
                    eng_sentence = ''
                j+=1
    elif ('en' in data['transcript']):
        for i, en in enumerate(data['transcript']['en']):
            en_text = en['text']
            en_start = en['start']  
            en_duration = en['duration']
            eng_sentence += ' ' + en_text
            if (is_sentence(eng_sentence)):
                eng_sentences.append(clean(eng_sentence))
                eng_sentence = ''
    return thai_sentences, eng_sentences

def _compare_print(ths, ens):
    for th,en in zip(ths,ens):
        print(en)
        print(th)
        print('-'*10)

def read_transcripts(channel):
    os.makedirs(f'all_transcript/{channel}/format', exist_ok=True)
    for file_path in glob.glob(f'all_transcript/{channel}/transcripts/*.json'):
        with open(file_path, 'r') as file:
            json_name = os.path.splitext(os.path.basename(file_path))[0]
            data = json.load(file)
            ths, ens = format(data)
            if len(ths) != len(ens):
                max_len = max(len(ths), len(ens))
                ths += [''] * (max_len - len(ths)) 
                ens += [''] * (max_len - len(ens)) 
            df = pd.DataFrame({'en': ens, 'th': ths})
            df.to_csv(f'all_transcript/{channel}/format/{json_name}.csv',index=False)

In [77]:
read_transcripts('khan')

In [64]:
# with open('all_transcript/ted-ed/transcripts/-EG6rqA2vvA.json', 'r') as file:
#             data = json.load(file)
#             ths, ens = format(data)

0
1
